## Spark Core: Resilient Distributed Dataset

- Son estructuras de datos que contienen elementos que pueden ser manipulados en paralelo
    - Distribuidos en un conjunto de máquinas/nodos de computación
    - Las transformaciones de datos se pueden recrear/regenerar en caso de fallo de nodo/máquina o corrupción de memoria
    
- Abstracciones de bajo nivel en Spark 2.x
    - Datasets, Dataframes (API de programación, abastracciones de alto nivel)
        - dataset.rdd()
        - SparkSession.createDataset(rdd), SparkSession.createDataFrame(rdd)
    - Todos los datos son representados como RDDs

- RDDs en Scala -> Scala Collection Library

## Fundamentos de RDDs

- Hay dos tipos de operaciones con RDD
    - Transformaciones: define las operaciones
        - Creación de DAGs(Direct Acyclic Graph): define cómo se va a ejecutar la operación.
        - Evaluaciones "en diferido/vagas/perezosas" (lazy): no se ejecutará esa transformación hasta que no se ejecute un evento adicional
        - No devuelven valores
    - Acciones: ejecuta operaciones y devuelve resultados
        - Ejecutan las transformaciones y la propia acción
        - Devuelven un valor
    - Son tolerantes a fallos
    - Se pueden cachear
    - Ejemplo de flujo
        - Hadoop RDD > RDD Filtrado > RDD Mapeado > RDD Reducido

- Los RDDs son inmutables

- Existen tres métodos para crear RDDs
    - Paralelizar una colección ya existente
    - Referenciar un Dataset
    - Usar una transformación de un RDD existente

- Cuando se crea un RDD se crea un DAG
    - Las transformaciones actualizan el DAG pero no se "ejecuta" nada, hasta que asignemos en el código una acción

- Los datasets pueden usarse desde cualquier tipo de almacenamiento soportado por Hadoop: HDFS, Cassandra, Hbase, Amazon S3, etc.

- Tipo de ficheros soportados
    - Texto
    - Ficheros secuenciales (SequenceFiles)
    - Hadoop (InputFormat)

- Ejemplo de creación de un RDD a partir de una colección existente

In [3]:
# Spark-shell
# val data = 1 to 10000
# val miRdd = sc.parallelize(data)
# val filtered = miRdd.filter(x => x<10)
# filtered.collect()

In [ ]:
# PySpark

In [1]:
data = range(1,10001)

In [2]:
miRdd = sc.parallelize(data) # crea rdd

In [3]:
filtered = miRdd.filter(lambda x:x<10) # lambda <argumento de entrada>:<función>, devuelve un rdd nuevo

In [5]:
filtered.collect() # ejecuta la acción que hace que se realicen las transformaciones

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [6]:
filtered.toDebugString()

b'(12) PythonRDD[1] at collect at <ipython-input-4-21dab2cd9579>:1 []\n |   ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262 []'

| Transformation|	Meaning
|---|---|
|map(func)|	Return a new distributed dataset formed by passing each element of the source through a function func.|
|filter(func)|	Return a new dataset formed by selecting those elements of the source on which func returns true.|
|flatMap(func)|	Similar to map, but each input item can be mapped to 0 or more output items (so func should return a Seq rather than a single item).|
|mapPartitions(func)|	Similar to map, but runs separately on each partition (block) of the RDD, so func must be of type Iterator<T> => Iterator<U> when running on an RDD of type T.|
|mapPartitionsWithIndex(func)|	Similar to mapPartitions, but also provides func with an integer value representing the index of the partition, so func must be of type (Int, Iterator<T>) => |Iterator<U> when running on an RDD of type T.
|sample(withReplacement, fraction, seed)|	Sample a fraction fraction of the data, with or without replacement, using a given random number generator seed.|
|union(otherDataset)|	Return a new dataset that contains the union of the elements in the source dataset and the argument.|
|intersection(otherDataset)|	Return a new RDD that contains the intersection of elements in the source dataset and the argument.|
|distinct([numPartitions]))|	Return a new dataset that contains the distinct elements of the source dataset.|
|groupByKey([numPartitions])|	When called on a dataset of (K, V) pairs, returns a dataset of (K, Iterable<V>) pairs.Note: If you are grouping in order to perform an aggregation (such as a sum or average) over each key, using reduceByKey or aggregateByKey will yield much better performance.Note: By default, the level of parallelism in the output depends on the number of partitions of the parent RDD. You can pass an optional numPartitions argument to set a different number of tasks.|
|reduceByKey(func, [numPartitions])|	When called on a dataset of (K, V) pairs, returns a dataset of (K, V) pairs where the values for each key are aggregated using the given reduce function func, which |must be of type (V,V) => V. Like in groupByKey, the number of reduce tasks is configurable through an optional second argument.
|aggregateByKey(zeroValue)(seqOp, combOp, [numPartitions])|	When called on a dataset of (K, V) pairs, returns a dataset of (K, U) pairs where the values for each key are aggregated using the given |combine functions and a neutral "zero" value. Allows an aggregated value type that is different than the input value type, while avoiding unnecessary allocations. Like in groupByKey, the number of reduce tasks is configurable through an optional second argument.
|sortByKey([ascending], [numPartitions])|	When called on a dataset of (K, V) pairs where K implements Ordered, returns a dataset of (K, V) pairs sorted by keys in ascending or descending |order, as specified in the boolean ascending argument.
|join(otherDataset, [numPartitions])|	When called on datasets of type (K, V) and (K, W), returns a dataset of (K, (V, W)) pairs with all pairs of elements for each key. Outer joins are supported |through leftOuterJoin, rightOuterJoin, and fullOuterJoin.
|cogroup(otherDataset, [numPartitions])|	When called on datasets of type (K, V) and (K, W), returns a dataset of (K, (Iterable<V>, Iterable<W>)) tuples. This operation is also called groupWith.|
|cartesian(otherDataset)|	When called on datasets of types T and U, returns a dataset of (T, U) pairs (all pairs of elements).|
|pipe(command, [envVars])|	Pipe each partition of the RDD through a shell command, e.g. a Perl or bash script. RDD elements are written to the process's stdin and lines output to its stdout are |returned as an RDD of strings.
|coalesce(numPartitions)|	Decrease the number of partitions in the RDD to numPartitions. Useful for running operations more efficiently after filtering down a large dataset.|
|repartition(numPartitions)|	Reshuffle the data in the RDD randomly to create either more or fewer partitions and balance it across them. This always shuffles all data over the network.|
|repartitionAndSortWithinPartitions(partitioner)|	Repartition the RDD according to the given partitioner and, within each resulting partition, sort records by their keys. This is more efficient |than calling repartition and then sorting within each partition because it can push the sorting down into the shuffle machinery.


|Action|	Meaning|
|---|---|
|reduce(func)|	Aggregate the elements of the dataset using a function func (which takes two arguments and returns one). The function should be commutative and associative so that it can be computed correctly in parallel.|
|collect()|	Return all the elements of the dataset as an array at the driver program. This is usually useful after a filter or other operation that returns a sufficiently small subset of the data.|
|count()|	Return the number of elements in the dataset.|
|first()|	Return the first element of the dataset (similar to take(1)).|
|take(n)|	Return an array with the first n elements of the dataset.|
|takeSample(withReplacement, num, [seed])|	Return an array with a random sample of num elements of the dataset, with or without replacement, optionally pre-specifying a random number generator seed.
|takeOrdered(n, [ordering])|	Return the first n elements of the RDD using either their natural order or a custom comparator.|
|saveAsTextFile(path)|	Write the elements of the dataset as a text file (or set of text files) in a given directory in the local filesystem, HDFS or any other Hadoop-supported file system. Spark will call toString on each element to convert it to a line of text in the file.|
|saveAsSequenceFile(path)|(Java and Scala)	Write the elements of the dataset as a Hadoop SequenceFile in a given path in the |local filesystem, HDFS or any other Hadoop-supported file system. This is available on RDDs of key-value pairs that implement Hadoop's Writable interface. In Scala, it is also available on types that are implicitly convertible to Writable (Spark includes conversions for basic types like Int, Double, String, etc).|
|saveAsObjectFile(path)|(Java and Scala)	Write the elements of the dataset in a simple format using Java serialization, |which can then be loaded using SparkContext.objectFile().|
|countByKey()|	Only available on RDDs of type (K, V). Returns a hashmap of (K, Int) pairs with the count of each key.|
|foreach(func)|	Run a function func on each element of the dataset. This is usually done for side effects such as updating an Accumulator or interacting with external storage systems. Note: modifying variables other than Accumulators outside of the foreach() may result in undefined behavior. See Understanding closures for more details.|

## Persistencia y RDDS

- Cada nodo almacena las particiones en la memoria caché(cálculos en memoria)

- Reutilización de las particiones enotras acciones sobre el RDD (rapidez y eficiencia)

- Dos métodos: persis() | cache() -> persist() + MEMORY_ONLY



|Storage Level|	Meaning|
|---|---|
|MEMORY_ONLY|	Store RDD as deserialized Java objects in the JVM. If the RDD does not fit in memory, |some partitions will not be cached and will be recomputed on the fly each time they're needed. This is the default level.
|MEMORY_AND_DISK|	Store RDD as deserialized Java objects in the JVM. If the RDD does not fit in |memory, store the partitions that don't fit on disk, and read them from there when they're needed.
|MEMORY_ONLY_SER| (Java and Scala)	Store RDD as serialized Java objects (one byte array per partition). This is |generally more space-efficient than deserialized objects, especially when using a fast serializer, but more CPU-intensive to read.|
|MEMORY_AND_DISK_SER| (Java and Scala)	Similar to MEMORY_ONLY_SER, but spill partitions that don't fit in memory to disk |instead of recomputing them on the fly each time they're needed.|
|DISK_ONLY|	Store the RDD partitions only on disk.|
|MEMORY_ONLY_2, MEMORY_AND_DISK_2, etc.|	Same as the levels above, but replicate each partition on two cluster nodes.|
|OFF_HEAP (experimental)|	Similar to MEMORY_ONLY_SER, but store the data in off-heap memory. This requires off-heap memory to be enabled.|


## Recomendaciones sobre cuál nivel usar? (IBM)

- Por defecto usar MEMORY_ONLY y si no es posible utlilizarlo por restricciones computacionales, utilizar MEMORY_ONLY_SER que implica utilizar una líbrearía en conjunto de serialización de memoria en disco, muchas más rápidas de las que vienen por defecto.

## Acumuladores y variables Broadcast

- El driver envía una copia separada de las variables a los nodos.

- Implementar el concepto de estado compartido (share state)
    - Broadcast variable. La variable se envía una sola vez a todos los nodos
        - Es de solo lectura (inmutable)
        - sc.broadcast(value)->devuelve la referencia a la variable
    - Accumulator
        - Contadores y sumas (operaciones de agregación): add() | +=
        - Solo el driver puede leer el valor de la variable acumulador
        - Solo los nodos pueden agregar información al acumulador(sumas)
        - Soporte para tipos numéricos (aunque se pueden implementar tipos específicos)
        
- Ejemplo en Scala

In [7]:
# Spark-shell
# val accum = sc.longAccumulator("My Accumulator")
# sc.parallelize(Array(1,2,3,4)).foreach(x => accum.add(x))
# accum.value

- Ejemplo en pyspark

In [14]:
accum = sc.accumulator(0)

In [15]:
accum

Accumulator<id=2, value=0>

In [17]:
sc.parallelize([1,2,3,4]).foreach(lambda x:accum.add(x))

In [18]:
accum.value

10

## Dataframes y Datasets

- Desventajas del API de RDD
    - % Alto de datos estructurados / semiestructurados (JSON, CSV, Avro, Parquet, Hive)-> Uso intensivo de tuplas en RDDs (_1,_2,[0],[1]...)
    - Las transformaciones funcionales no son intuitivas (map/reduce, etc.)
    
Ejemplo:

In [ ]:
pdata.map(lambda x: (x.dept, [x.age, 1]))
     .reduceByKey(lambda x,y: [x[0] + y[0], x[1] + y[1]])
     .map(lambda x: [x[0], x[1][0] / x[1][1]])
     .collect()

Se podría traducir como:

In [ ]:
pdata.groupBy("dept").avg("age")

Mas sobre Datasets y Dataframes:

- Colección distribuida de datos agrupadas en columnas con nombre
    - RDD con esquema de datos (SchemaRDD)
- Funciones específicas de dominio diseñadas para las tareas comunes
    - Metadata
    - Sampling
    - Filtrado, Agregación, Union, ...
    - UDFs
- Disponibles en Python, Scala, Java y R (SparkR)

Los datasets y dataframes mejoran la legibilidad del código y mejoran la eficiencia de ejución.

## Optimización y soporte

Versiones 1.X

    - Optimizadas: Catalyst Optimizer > RDDs
        - Spark SQL / Dataframes: 

    - No optimizadas: RDDs
        - Spark Streaming / MLlib (RDD based) / GraphX

Versiones 2.x

    - Optimizadas: Catalyst Optimizer > RDDs
        - SparkSQL / DataFrames / Datasets
        - MLlib (DF based)
        - Structured Streaming
        - GraphFrames

    - No optimizadas: RDDs
        - Spark Streaming
        - MLlib (RDD based)
        - GraphX